In [1]:
import pandas as pd
import numpy as np
current_state = np.random.get_state()
np.random.set_state(current_state)

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline

from scipy import stats

import os

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

C:\Users\sebar\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# Expanding column and row width for convenience

pd.options.display.max_columns = 200
pd.options.display.max_rows = 200

In [3]:
# Loading the dataset

df = pd.read_csv('comtrade.csv')

In [4]:
# High-level glance

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18844 entries, 0 to 18843
Data columns (total 35 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Classification          18844 non-null  object 
 1   Year                    18844 non-null  int64  
 2   Period                  18844 non-null  int64  
 3   Period Desc.            18844 non-null  int64  
 4   Aggregate Level         18844 non-null  int64  
 5   Is Leaf Code            18844 non-null  int64  
 6   Trade Flow Code         18844 non-null  int64  
 7   Trade Flow              18844 non-null  object 
 8   Reporter Code           18844 non-null  int64  
 9   Reporter                18844 non-null  object 
 10  Reporter ISO            18844 non-null  object 
 11  Partner Code            18844 non-null  int64  
 12  Partner                 18844 non-null  object 
 13  Partner ISO             18844 non-null  object 
 14  2nd Partner Code        0 non-null    

In [5]:
print('This dataset has {} rows and {} columns.'.format(df.shape[0], df.shape[1]))

This dataset has 18844 rows and 35 columns.


In [6]:
# Showing the first ten observations

df.head(10)

,Classification,Year,Period,Period Desc.,Aggregate Level,Is Leaf Code,Trade Flow Code,Trade Flow,Reporter Code,Reporter,Reporter ISO,Partner Code,Partner,Partner ISO,2nd Partner Code,2nd Partner,2nd Partner ISO,Customs Proc. Code,Customs,Mode of Transport Code,Mode of Transport,Commodity Code,Commodity,Qty Unit Code,Qty Unit,Qty,Alt Qty Unit Code,Alt Qty Unit,Alt Qty,Netweight (kg),Gross weight (kg),Trade Value (US$),CIF Trade Value (US$),FOB Trade Value (US$),Flag
0,H5,2017,2017,2017,2,0,1,Import,156,China,CHN,604,Peru,PER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01,Animals; live,1,No Quantity,0.0,NaN,NaN,NaN,0.0,NaN,425616,NaN,NaN,0
1,H5,2018,2018,2018,2,0,1,Import,156,China,CHN,604,Peru,PER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01,Animals; live,1,No Quantity,0.0,NaN,NaN,NaN,0.0,NaN,1165296,NaN,NaN,0
2,H5,2017,2017,2017,4,0,1,Import,156,China,CHN,604,Peru,PER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0106,"Animals; live, n.e.c. in chapter 01",5,Number of items,276282.0,NaN,NaN,NaN,9514.0,NaN,425616,NaN,NaN,0
3,H5,2018,2018,2018,4,0,1,Import,156,China,CHN,604,Peru,PER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0106,"Animals; live, n.e.c. in chapter 01",5,Number of items,434386.0,NaN,NaN,NaN,10935.0,NaN,1165296,NaN,NaN,0
4,H5,2017,2017,2017,6,1,1,Import,156,China,CHN,604,Peru,PER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,010619,"Mammals; live, other than primates, whales, do...",5,Number of items,24.0,NaN,NaN,NaN,160.0,NaN,19058,NaN,NaN,0
5,H5,2018,2018,2018,6,1,1,Import,156,China,CHN,604,Peru,PER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,010619,"Mammals; live, other than primates, whales, do...",5,Number of items,63.0,NaN,NaN,NaN,370.0,NaN,75909,NaN,NaN,0
6,H5,2017,2017,2017,6,1,1,Import,156,China,CHN,604,Peru,PER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,010620,Reptiles; live (including snakes and turtles),5,Number of items,276258.0,NaN,NaN,NaN,9354.0,NaN,406558,NaN,NaN,0
7,H5,2018,2018,2018,6,1,1,Import,156,China,CHN,604,Peru,PER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,010620,Reptiles; live (including snakes and turtles),5,Number of items,434260.0,NaN,NaN,NaN,9976.0,NaN,953894,NaN,NaN,0
8,H5,2018,2018,2018,6,1,1,Import,156,China,CHN,604,Peru,PER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,010639,"Birds; live, other than birds of prey, Psittac...",5,Number of items,7.0,NaN,NaN,NaN,84.0,NaN,106456,NaN,NaN,0
9,H5,2018,2018,2018,6,1,1,Import,156,China,CHN,604,Peru,PER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,010690,"Animals; live, n.e.c. in chapter 01, other tha...",5,Number of items,56.0,NaN,NaN,NaN,505.0,NaN,29037,NaN,NaN,0


In [7]:
# Missing values pt. 1

df.isnull().sum()

Classification                0
Year                          0
Period                        0
Period Desc.                  0
Aggregate Level               0
Is Leaf Code                  0
Trade Flow Code               0
Trade Flow                    0
Reporter Code                 0
Reporter                      0
Reporter ISO                  0
Partner Code                  0
Partner                       0
Partner ISO                   0
2nd Partner Code          18844
2nd Partner               18844
2nd Partner ISO           18844
Customs Proc. Code        18844
Customs                   18844
Mode of Transport Code    18844
Mode of Transport         18844
Commodity Code                0
Commodity                     0
Qty Unit Code                 0
Qty Unit                      0
Qty                         914
Alt Qty Unit Code         18844
Alt Qty Unit              18844
Alt Qty                   18844
Netweight (kg)             1119
Gross weight (kg)         18844
Trade Va

In [8]:
# Missing values pt. 2

def missing_values(df):
    mis_val= df.isnull().sum()
    
    mis_val_percent= df.isnull().sum() / len(df)
    
    mis_val_table=pd.concat([mis_val, mis_val_percent], axis=1)
    
    mis_val_table = mis_val_table.reset_index()
    
    mis_val_table_rename = mis_val_table.rename(columns= {'index': 'Column', 0: 'Missing Values', 1: '% of Total Values'})
    
    mis_val_table_rename = mis_val_table_rename[mis_val_table_rename.iloc[:,1] != 0].sort_values('% of Total Values',
                                                                                                ascending=False).round(2)
    
    print('Your table has ' + str(df.shape[1]) + ' columns.\nThere are ' +str(mis_val_table_rename.shape[0]) + ' columns with missing values.' )
    
    return mis_val_table_rename

missing_values(df)

Your table has 35 columns.
There are 15 columns with missing values.


,Column,Missing Values,% of Total Values
14,2nd Partner Code,18844,1.00
15,2nd Partner,18844,1.00
16,2nd Partner ISO,18844,1.00
17,Customs Proc. Code,18844,1.00
18,Customs,18844,1.00
19,Mode of Transport Code,18844,1.00
20,Mode of Transport,18844,1.00
26,Alt Qty Unit Code,18844,1.00
27,Alt Qty Unit,18844,1.00
28,Alt Qty,18844,1.00


In [9]:
# Since we're interested in non-FOB/CIF Trade Value (US$) and it's not included above, we can drop all columns with
# null values.

df.dropna(axis=1, inplace=True)

In [10]:
# Checking shape again

print('This dataset has {} rows and {} columns.'.format(df.shape[0], df.shape[1]))

This dataset has 18844 rows and 20 columns.


In [11]:
df.head()

,Classification,Year,Period,Period Desc.,Aggregate Level,Is Leaf Code,Trade Flow Code,Trade Flow,Reporter Code,Reporter,Reporter ISO,Partner Code,Partner,Partner ISO,Commodity Code,Commodity,Qty Unit Code,Qty Unit,Trade Value (US$),Flag
0,H5,2017,2017,2017,2,0,1,Import,156,China,CHN,604,Peru,PER,01,Animals; live,1,No Quantity,425616,0
1,H5,2018,2018,2018,2,0,1,Import,156,China,CHN,604,Peru,PER,01,Animals; live,1,No Quantity,1165296,0
2,H5,2017,2017,2017,4,0,1,Import,156,China,CHN,604,Peru,PER,0106,"Animals; live, n.e.c. in chapter 01",5,Number of items,425616,0
3,H5,2018,2018,2018,4,0,1,Import,156,China,CHN,604,Peru,PER,0106,"Animals; live, n.e.c. in chapter 01",5,Number of items,1165296,0
4,H5,2017,2017,2017,6,1,1,Import,156,China,CHN,604,Peru,PER,010619,"Mammals; live, other than primates, whales, do...",5,Number of items,19058,0


In [12]:
# 4813 unique commodity categories. We will simplify by filtering for 2-digit (aka simpler) commodity codes

df['Commodity Code'].nunique()

4813

In [13]:
# Filtering for 2-character commodity codes

mask = (df['Commodity Code'].str.len() <= 2)

# Applying the filter

df = df.loc[mask]

# Visualizing

df.tail()

,Classification,Year,Period,Period Desc.,Aggregate Level,Is Leaf Code,Trade Flow Code,Trade Flow,Reporter Code,Reporter,Reporter ISO,Partner Code,Partner,Partner ISO,Commodity Code,Commodity,Qty Unit Code,Qty Unit,Trade Value (US$),Flag
18820,H4,2015,2015,2015,2,0,2,Export,156,China,CHN,604,Peru,PER,97,Works of art; collectors' pieces and antiques,1,No Quantity,829803,0
18821,H4,2016,2016,2016,2,0,1,Import,156,China,CHN,604,Peru,PER,97,Works of art; collectors' pieces and antiques,1,No Quantity,22928,0
18831,H4,2016,2016,2016,2,0,2,Export,156,China,CHN,604,Peru,PER,99,Commodities not specified according to kind,1,No Quantity,1849346,0
18832,H4,2016,2016,2016,2,0,1,Import,156,China,CHN,604,Peru,PER,99,Commodities not specified according to kind,1,No Quantity,189343,0
18833,H4,2015,2015,2015,2,0,2,Export,156,China,CHN,604,Peru,PER,99,Commodities not specified according to kind,1,No Quantity,204175,0


In [14]:
# df = df[(df['Commodity Code'] != '99') & (df['Commodity Code'] != '9999')]

# '99' is essentially a null-category. We'll drop it.

df = df[(df['Commodity Code'] != '99')]

# 95 unique categories

df['Commodity Code'].nunique()

95

In [15]:
df.shape

(660, 20)

In [16]:
# Separating imports and exports

imports = df[df['Trade Flow'] ==  'Import']

exports = df[df['Trade Flow'] ==  'Export']

In [17]:
# Grouping the values by commodity category and displaying 4-year average trade value (high-to-low)

comm_imports = imports.groupby('Commodity').mean().sort_values(by= 'Trade Value (US$)', ascending = False)

comm_exports = exports.groupby('Commodity').mean().sort_values(by= 'Trade Value (US$)', ascending = False)

pd.set_option('display.max_colwidth', 300)

In [18]:
print('Top 10 Chinese imports from Peru by Trade Value(US$)(2015-19)')

comm_imports.head(10)

Top 10 Chinese imports from Peru by Trade Value(US$)(2015-19)


,Year,Period,Period Desc.,Aggregate Level,Is Leaf Code,Trade Flow Code,Reporter Code,Partner Code,Qty Unit Code,Trade Value (US$),Flag
Commodity,,,,,,,,,,,
"Ores, slag and ash",2016.5,2016.5,2016.5,2.0,0.0,1.0,156.0,604.0,1.0,8.721363e+09,0.0
"Food industries, residues and wastes thereof; prepared animal fodder",2016.5,2016.5,2016.5,2.0,0.0,1.0,156.0,604.0,1.0,1.051283e+09,0.0
Copper and articles thereof,2016.5,2016.5,2016.5,2.0,0.0,1.0,156.0,604.0,1.0,1.003359e+09,1.0
"Fruit and nuts, edible; peel of citrus fruit or melons",2016.5,2016.5,2016.5,2.0,0.0,1.0,156.0,604.0,1.0,2.091290e+08,1.0
"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",2016.5,2016.5,2016.5,2.0,0.0,1.0,156.0,604.0,1.0,1.084123e+08,0.0
"Meat, fish or crustaceans, molluscs or other aquatic invertebrates; preparations thereof",2016.5,2016.5,2016.5,2.0,0.0,1.0,156.0,604.0,1.0,9.632819e+07,0.0
Wood and articles of wood; wood charcoal,2016.5,2016.5,2016.5,2.0,0.0,1.0,156.0,604.0,1.0,4.761761e+07,0.0
Zinc and articles thereof,2016.5,2016.5,2016.5,2.0,0.0,1.0,156.0,604.0,1.0,4.756393e+07,0.0
"Wool, fine or coarse animal hair; horsehair yarn and woven fabric",2016.5,2016.5,2016.5,2.0,0.0,1.0,156.0,604.0,1.0,4.235855e+07,0.0


In [19]:
print('Bottom 10 Chinese imports from Peru by Trade Value(US$)(2015-19)')

comm_imports.tail(10)

Bottom 10 Chinese imports from Peru by Trade Value(US$)(2015-19)


,Year,Period,Period Desc.,Aggregate Level,Is Leaf Code,Trade Flow Code,Reporter Code,Partner Code,Qty Unit Code,Trade Value (US$),Flag
Commodity,,,,,,,,,,,
"Paper and paperboard; articles of paper pulp, of paper or paperboard",2016.5,2016.5,2016.5,2.0,0.0,1.0,156.0,604.0,1.0,2871.25,1.000000
"Manufactures of straw, esparto or other plaiting materials; basketware and wickerwork",2017.5,2017.5,2017.5,2.0,0.0,1.0,156.0,604.0,1.0,2650.50,2.000000
Miscellaneous manufactured articles,2016.5,2016.5,2016.5,2.0,0.0,1.0,156.0,604.0,1.0,2528.75,2.000000
Musical instruments; parts and accessories of such articles,2017.0,2017.0,2017.0,2.0,0.0,1.0,156.0,604.0,1.0,1551.00,2.666667
"Feathers and down, prepared; and articles made of feather or of down; artificial flowers; articles of human hair",2017.0,2017.0,2017.0,2.0,0.0,1.0,156.0,604.0,1.0,1160.00,0.000000
Sugars and sugar confectionery,2017.0,2017.0,2017.0,2.0,0.0,1.0,156.0,604.0,1.0,521.00,0.000000
Tin; articles thereof,2017.5,2017.5,2017.5,2.0,0.0,1.0,156.0,604.0,1.0,378.00,0.000000
Explosives; pyrotechnic products; matches; pyrophoric alloys; certain combustible preparations,2018.0,2018.0,2018.0,2.0,0.0,1.0,156.0,604.0,1.0,200.00,0.000000
Fertilizers,2016.0,2016.0,2016.0,2.0,0.0,1.0,156.0,604.0,1.0,167.00,0.000000


In [20]:
print('Top 10 Chinese exports to Peru by Trade Value(US$)(2015-19)')

comm_exports.head(10)

Top 10 Chinese exports to Peru by Trade Value(US$)(2015-19)


,Year,Period,Period Desc.,Aggregate Level,Is Leaf Code,Trade Flow Code,Reporter Code,Partner Code,Qty Unit Code,Trade Value (US$),Flag
Commodity,,,,,,,,,,,
"Electrical machinery and equipment and parts thereof; sound recorders and reproducers; television image and sound recorders and reproducers, parts and accessories of such articles",2016.5,2016.5,2016.5,2.0,0.0,2.0,156.0,604.0,1.0,1.214890e+09,2.0
"Nuclear reactors, boilers, machinery and mechanical appliances; parts thereof",2016.5,2016.5,2016.5,2.0,0.0,2.0,156.0,604.0,1.0,7.943537e+08,2.0
"Vehicles; other than railway or tramway rolling stock, and parts and accessories thereof",2016.5,2016.5,2016.5,2.0,0.0,2.0,156.0,604.0,1.0,5.434561e+08,2.0
Iron and steel,2016.5,2016.5,2016.5,2.0,0.0,2.0,156.0,604.0,1.0,4.324033e+08,1.0
Iron or steel articles,2016.5,2016.5,2016.5,2.0,0.0,2.0,156.0,604.0,1.0,3.546047e+08,2.0
Plastics and articles thereof,2016.5,2016.5,2016.5,2.0,0.0,2.0,156.0,604.0,1.0,3.235308e+08,2.0
"Furniture; bedding, mattresses, mattress supports, cushions and similar stuffed furnishings; lamps and lighting fittings, n.e.c.; illuminated signs, illuminated name-plates and the like; prefabricated buildings",2016.5,2016.5,2016.5,2.0,0.0,2.0,156.0,604.0,1.0,2.258472e+08,1.0
"Toys, games and sports requisites; parts and accessories thereof",2016.5,2016.5,2016.5,2.0,0.0,2.0,156.0,604.0,1.0,2.257129e+08,2.0
Apparel and clothing accessories; not knitted or crocheted,2016.5,2016.5,2016.5,2.0,0.0,2.0,156.0,604.0,1.0,1.807743e+08,2.0


In [21]:
print('Bottom 10 Chinese exports to Peru by Trade Value(US$)(2015-19)')

comm_exports.tail(10)

Bottom 10 Chinese exports to Peru by Trade Value(US$)(2015-19)


,Year,Period,Period Desc.,Aggregate Level,Is Leaf Code,Trade Flow Code,Reporter Code,Partner Code,Qty Unit Code,Trade Value (US$),Flag
Commodity,,,,,,,,,,,
Animal originated products; not elsewhere specified or included,2016.500000,2016.500000,2016.500000,2.0,0.0,2.0,156.0,604.0,1.0,46058.000000,0.0
Arms and ammunition; parts and accessories thereof,2016.500000,2016.500000,2016.500000,2.0,0.0,2.0,156.0,604.0,1.0,21602.750000,1.0
Cork and articles of cork,2016.500000,2016.500000,2016.500000,2.0,0.0,2.0,156.0,604.0,1.0,21058.500000,0.0
Nickel and articles thereof,2016.500000,2016.500000,2016.500000,2.0,0.0,2.0,156.0,604.0,1.0,16526.250000,0.0
Furskins and artificial fur; manufactures thereof,2016.500000,2016.500000,2016.500000,2.0,0.0,2.0,156.0,604.0,1.0,8546.250000,0.0
Vegetable plaiting materials; vegetable products not elsewhere specified or included,2017.500000,2017.500000,2017.500000,2.0,0.0,2.0,156.0,604.0,1.0,2707.500000,0.0
Pulp of wood or other fibrous cellulosic material; recovered (waste and scrap) paper or paperboard,2015.000000,2015.000000,2015.000000,2.0,0.0,2.0,156.0,604.0,1.0,2630.000000,0.0
Lead and articles thereof,2016.666667,2016.666667,2016.666667,2.0,0.0,2.0,156.0,604.0,1.0,1178.666667,0.0
Cereals,2016.500000,2016.500000,2016.500000,2.0,0.0,2.0,156.0,604.0,1.0,1157.000000,0.0


In [22]:
# Filters for future visualizations

import_top_10 = comm_imports[comm_imports['Trade Value (US$)'] >= 3.740093e+07]
import_bottom_10 = comm_imports[comm_imports['Trade Value (US$)'] <= 2871.25]
export_top_10 = comm_exports[comm_exports['Trade Value (US$)'] >= 1.705532e+08]
export_bottom_10 = comm_exports[comm_exports['Trade Value (US$)'] <= 46058.000000]